In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["HF_API_KEY"]=os.getenv("HF_API_KEY")


In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [7]:
#Builtin Tools:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [19]:
wikiAPI = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxviAPI = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)

arxiv = ArxivQueryRun(api_wrapper=arxviAPI)
wikipedia =WikipediaQueryRun(api_wrapper=wikiAPI)

In [22]:
# Custom tools: RAG tool
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma


In [23]:
loader = WebBaseLoader(web_path="https://oig.hhs.gov/oig-policies/official-site/")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap=100)
splitted_docs = splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
db = Chroma.from_documents(embedding=embeddings,documents=splitted_docs)

In [17]:
retriver = db.as_retriever()

In [24]:
from langchain.tools.retriever import create_retriever_tool

retriver_tool = create_retriever_tool(retriver,"RAG tool","Description: Searching info From US Health department")

In [25]:
# Creating chains of tools:

tools = [arxiv,wikipedia,retriver_tool]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI agent. Solve the user's query with given tools."),
    MessagesPlaceholder("chat_history"),
    ("user", "{input_Text}")
])



In [40]:
from langchain.agents import initialize_agent,AgentType
agent = initialize_agent(tools=tools,llm=llm,agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [ ]:
agent.run("what are the frauds faced by US Health department")



> Entering new AgentExecutor chain...
Question: What are the frauds faced by the US Health department

Thought: To find information about the frauds faced by the US Health department, I should search for relevant data from a reliable source. The RAG tool, which searches information from the US Health department, would be a good starting point.

Action:
```
{
  "action": "RAG tool",
  "action_input": "US Health department frauds"
}
```


Observation: Grant Fraud
                



                    Medicaid Fraud Control Units
                



                    Medicare Fraud Strike Force
                



                    Operation CARE
                



                    State False Claims Act Reviews
                



                    Submit a Hotline Complaint
                



                    Whistleblower Protection Coordinator
                





Compliance

Grant Fraud
                



                    Medicaid Fraud Control Units
          

'The US Health department faces various types of fraud, including Grant Fraud, Medicaid Fraud, and Medicare Fraud. To combat these frauds, the department has established measures such as Medicaid Fraud Control Units, Medicare Fraud Strike Force, Operation CARE, and State False Claims Act Reviews. Additionally, there are mechanisms for submitting complaints and protecting whistleblowers.'

In [43]:
agent.run("what is a theory of Monster zero in monsterverse movies series")



> Entering new AgentExecutor chain...
Question: What is a theory of Monster zero in Monsterverse movies series
Thought: To find the theory of Monster zero in Monsterverse movies series, I need to search for information about the Monsterverse and its related concepts. The best place to start would be to look up information on Wikipedia, which likely has a page about the Monsterverse and its related theories.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Monster Zero Monsterverse"
}
```

Observation: Page: Godzilla: King of the Monsters (2019 film)
Summary: Godzilla: King of the Monsters is a 2019 American monster film produced by Legendary Pictures and distributed by Warner Bros. Pictures. Co-wri
Thought:It seems like the previous search didn't yield the desired results. Let me try again with a more specific search query.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Monster Zero theory Monsterverse"
}
```


Observation: Page: King Ghidorah
Summary: King Ghid

'Agent stopped due to iteration limit or time limit.'